Calculate derivatives using talib

In [2]:
import pandas_ta as ta
import pandas as pd

def add_technical_indicators(filename):
    df = pd.read_csv(filename)

    # Reverse DataFrame
    df = df[::-1]

    # EMA
    df['ema_20'] = ta.ema(df['Close'], length=20)
    df['ema_50'] = ta.ema(df['Close'], length=50)

    # RSI
    df['rsi'] = ta.rsi(df['Close'], length=14)

    # MACD
    df['macd'] = ta.macd(df['Close'])['MACD_12_26_9']  # MACD line
    df['signal_line'] = ta.macd(df['Close'])['MACDs_12_26_9']   # Signal line

    # BBands
    bbands = ta.bbands(df['Close'], length=20, std=2)
    df['upper_band'] = bbands['BBU_20_2.0']
    df['middle_band'] = bbands['BBM_20_2.0']
    df['lower_band'] = bbands['BBL_20_2.0']

    return df

# Example usage:
filename = '/home/ghost-bot/Desktop/misc/extras/datax/NIFTY_5.csv'
df = add_technical_indicators(filename)
df = df[::-1]
df.head()

,Timestamp,Open,Max,Min,Volume,wvf,wvfl,rangeHigh,rangeHighl,Close,Entry,ema_20,ema_50,rsi,macd,signal_line,upper_band,middle_band,lower_band
0,1721987700,24836.45,24861.15,24830.40,8769215,0.085909,-0.400615,0.179519,-0.727505,24851.75,2,24809.638772,24743.926569,68.544879,27.420987,27.711352,24855.996145,24808.6600,24761.323855
1,1721987400,24834.25,24846.25,24832.05,11670418,0.068413,-0.340442,0.179519,-0.727505,24835.00,2,24805.206011,24739.525613,64.968277,26.869735,27.783943,24849.624006,24806.5825,24763.540994
2,1721987100,24848.00,24849.25,24830.75,12523130,0.073645,-0.352557,0.179519,-0.727505,24836.85,2,24802.069801,24735.628699,65.734839,27.480759,28.012495,24845.839123,24804.6100,24763.380877
3,1721986800,24820.10,24853.10,24819.30,13268557,0.119723,-0.368105,0.179519,-0.727505,24849.05,2,24798.408728,24731.497217,70.854187,27.672563,28.145429,24841.940281,24803.3050,24764.669719
4,1721986500,24827.65,24830.00,24810.75,12540918,0.071088,-0.274817,0.179519,-0.727505,24820.35,2,24793.078067,24726.699145,64.879467,26.272684,28.263646,24833.182952,24800.2325,24767.282048


In [ ]:
# Example usage
stockname = 'NIFTY'
tf = '5'
trade_type = 'sell'  # or 'buy'

Classify labels

calculate difference === feature engineering

In [3]:
import pandas as pd

# Calculate new columns
df['wvff'] = df['wvf'] - df['rangeHigh']
df['ema'] = df['ema_20'] - df['ema_50']
df['cema'] = df['Close'] - df['ema_20']
df['lowc'] = df['Close'] - df['lower_band']
df['macdf'] = df['macd'] - df['signal_line']
df['midlc'] = df['Close'] - df['middle_band']
df['wvflf'] = df['wvfl'] - df['rangeHighl']

data = df.drop(columns=['Volume','Open','Max','Min','wvf','rangeHigh','wvfl','rangeHighl','ema_20', 'ema_50','macd','signal_line','upper_band','lower_band','middle_band'])


row difference calculation

In [4]:
xx = data.copy()
xx = xx.drop(columns=['Entry'])
xx = xx[::-1]
df_diff = xx.diff().fillna(0)
df_diff = df_diff[::-1]
xx = xx[::-1]
df_diff['Entry'] = data['Entry']

for col in xx.columns[1:]:
    xx[col + '_diff'] = df_diff[col]

# data.drop(columns=['Entry_diff'])
xx['Entry'] = data['Entry']
# xx = xx
xx.tail()
# data = xx.copy()
# df_diff.head()

,Timestamp,Close,rsi,wvff,ema,cema,lowc,macdf,midlc,wvflf,Close_diff,rsi_diff,wvff_diff,ema_diff,cema_diff,lowc_diff,macdf_diff,midlc_diff,wvflf_diff,Entry
5269,1713153900,22268.75,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,-27.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
5270,1713153600,22296.00,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,-32.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
5271,1713153300,22328.05,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,3.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
5272,1713153000,22324.25,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,-0.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
5273,1713152700,22325.20,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


data preparation

In [6]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# data = pd.read_csv('/content/xyz.csv', index_col=0)
df = data
df = df.dropna()

x = df.drop(columns=['Timestamp', 'Entry'])

features = x.columns.tolist()
features.remove('Close')

scaler = MinMaxScaler(feature_range=(-100, 100))

sequences = []
labels = []
original_indices = []

for i in range(len(df) - 60):
    current_label = df['Entry'].iloc[i]
    if current_label != 2:
        original_indices.append(i)
        sequence_scaled = df[features].iloc[i:i+61].apply(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten(), axis=0).values.tolist()
        close_scaler = MinMaxScaler(feature_range=(-100, 100))
        close_scaled = close_scaler.fit_transform(df['Close'].iloc[i:i+61].values.reshape(-1, 1)).flatten().tolist()
        sequence_scaled = [[close] + scaled_features for close, scaled_features in zip(close_scaled, sequence_scaled)]
        sequences.append(sequence_scaled)
        labels.append(current_label)

sequences_reversed = [seq[::-1] for seq in sequences]

X = sequences_reversed
y = labels

corresponding_rows = [df.iloc[idx:idx+61] for idx in original_indices]

print(len(X[0][0]))
print(len(y))
print(X[0][0])


9
374
[-100.0, 45.465188244866056, -3.20667976713942, -100.0, 27.567013524482604, -11.831330317115743, 64.75191338161541, 22.622175512356065, -88.85392220373463]


In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# data = pd.read_csv('/content/xyz.csv', index_col=0)
df = df_diff
df = df.dropna()

x = df.drop(columns=['Timestamp', 'Entry'])

features = x.columns.tolist()
features.remove('Close')

scaler = MinMaxScaler(feature_range=(-100, 100))

sequences = []
labels = []
original_indices = []

for i in range(len(df) - 60):
    current_label = df['Entry'].iloc[i]
    if current_label != 2:
        original_indices.append(i)
        sequence_scaled = df[features].iloc[i:i+61].apply(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten(), axis=0).values.tolist()
        close_scaler = MinMaxScaler(feature_range=(-100, 100))
        close_scaled = close_scaler.fit_transform(df['Close'].iloc[i:i+61].values.reshape(-1, 1)).flatten().tolist()
        sequence_scaled = [[close] + scaled_features for close, scaled_features in zip(close_scaled, sequence_scaled)]
        sequences.append(sequence_scaled)
        labels.append(current_label)

sequences_reversed = [seq[::-1] for seq in sequences]

Xx = sequences_reversed
yx = labels

corresponding_rows = [df.iloc[idx:idx+61] for idx in original_indices]

print(len(Xx[0][0]))
print(len(yx))
print(Xx[0][0])


9
378
[20.424028268547126, 25.772039993832298, 62.3439229202842, 59.334368373683404, 10.210056620266652, 45.41403638578416, 44.74304858128474, 21.15356463183682, -9.55134913107741]


model prep

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()

model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(61, 9)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
# model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
# model.add(Dropout(0.2))
# model.add(Bidirectional(LSTM(64, return_sequences=True)))
# model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
# model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=32)

test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)


results and confusion matrix

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

X1 = np.array(X)
y1 = np.array(y)

xxc = model.predict(X1)
predictions = (xxc > 0.5).astype("int32")

conf_matrix1 = confusion_matrix(y1, predictions)
print("Confusion Matrix:")
print(conf_matrix1)

accuracy = accuracy_score(y1, predictions)
print("Accuracy:", accuracy)

precision = precision_score(y1, predictions, zero_division=1)
print("Precision:", precision)

recall = recall_score(y1, predictions, zero_division=1)
print("Recall:", recall)

# Printing predicted vs original values
# print("\nPredicted vs Original Values:")
# for i in range(len(predictions)):
#     print(f"Predicted: {predictions[i]}, Original: {y1[i]}, data: {df.iloc[original_indices[i]]['Timestamp']}")


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

X1 = np.array(Xx)
y1 = np.array(yx)

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

model2 = Sequential()
model2.add(LSTM(128, return_sequences=True, input_shape=(61, 9)))
model2.add(Dropout(0.2))
model2.add(LSTM(64, return_sequences=True))
model2.add(LSTM(64, return_sequences=True))
# model2.add(LSTM(64, return_sequences=True))
model2.add(LSTM(64, return_sequences=True))
model2.add(LSTM(64, return_sequences=True))
model2.add(LSTM(32))
model2.add(Dropout(0.2))
model2.add(Dense(1, activation='sigmoid'))

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model2.fit(X_train1, y_train1, epochs=20, batch_size=32)

test_loss, test_acc = model2.evaluate(X_test1, y_test1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

# all_weights = []
# all_biases = []
# for layer in model.layers:
#     if hasattr(layer, 'weights'):
#         weights = layer.get_weights()
#         if len(weights) > 0:
#             all_weights.append(weights[0])
#             all_biases.append(weights[1])

# print(all_weights)
# print(all_biases)


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

# Assuming Xx and yx are your input features and labels
X1 = np.array(Xx)  # Ensure X1 is a NumPy array
y1 = np.array(yx)  # Ensure y1 is a NumPy array

# Check the shape of the input data
print("Shape of X1:", X1.shape)

# Make predictions
xxl = model2.predict(X1)

# Convert predictions to binary class labels
predictionsl = (xxl > 0.5).astype("int32")

# Compute confusion matrix
conf_matrix1 = confusion_matrix(y1, predictionsl)
print("Confusion Matrix:")
print(conf_matrix1)

# Calculate accuracy
accuracy = accuracy_score(y1, predictionsl)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(y1, predictionsl, zero_division=1)
print("Precision:", precision)

# Calculate recall
recall = recall_score(y1, predictionsl, zero_division=1)
print("Recall:", recall)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Determine the minimum length among the sequences
min_length = min(len(xxc), len(xxl))
print("Minimum length:", min_length)

# Truncate both sequences to the minimum length
xxc_truncated = xxc[:min_length]
xxl_truncated = xxl[:min_length]

# Assuming y is a 1D array
yy = y

# Convert NumPy arrays to DataFrame columns
xx = pd.DataFrame({'s': xxc_truncated[:, 0], 'l': xxl_truncated[:, 0]})

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(xx, yy, test_size=0.6, random_state=42)

# Initialize and train the logistic regression model
modeld = LogisticRegression()
modeld.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = modeld.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

conf_matrix = confusion_matrix(y_test, y_pred)

# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix)


In [ ]:
from keras.models import save_model
import joblib
import os

# Function to construct paths based on parameters
def construct_paths(stockname, tf, trade_type, base_dir='./models'):
    if trade_type == 'sell':
        base_path = os.path.join(base_dir, f'nifty_models_{tf}min_{trade_type}')
        bilstm_model_file = f'{stockname.upper()}_{tf}_bilstm_{trade_type}_model.keras'
        lstm_model_file = f'{stockname.upper()}_{tf}_lstm_{trade_type}_model.keras'
        log_model_file = f'{stockname.upper()}_{tf}_log_{trade_type}_model.joblib'
    else:
        base_path = os.path.join(base_dir, f'nifty_models_{tf}min')
        bilstm_model_file = f'{stockname.upper()}_{tf}_bilstm_model.keras'
        lstm_model_file = f'{stockname.upper()}_{tf}_lstm_model.keras'
        log_model_file = f'{stockname.upper()}_{tf}_log_model.joblib'
    
    bilstm_model_path = os.path.join(base_path, bilstm_model_file)
    lstm_model_path = os.path.join(base_path, lstm_model_file)
    log_model_path = os.path.join(base_path, log_model_file)
    
    return bilstm_model_path, lstm_model_path, log_model_path

# Function to save models
def save_models(stockname, tf, trade_type, bilstm_model, lstm_model, log_model):
    bilstm_model_path, lstm_model_path, log_model_path = construct_paths(stockname, tf, trade_type)
    
    try:
        save_model(bilstm_model, bilstm_model_path)
        save_model(lstm_model, lstm_model_path)
        joblib.dump(log_model, log_model_path)
        print("Models saved successfully.")
    except Exception as e:
        print(f"Error saving models: {e}")

# Save models
save_models(stockname, tf, trade_type, model, model2, modeld)
